In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
data_dir = '..'
project_name = 'V2020_Rete_Sardegna_2021_06_03cr'
load_name = 'EqX_MIMC_I2201TR1_____LOAD____'
dP, dQ = 0.5, 0.0
data_file = '{}_{}_{:.1f}_{:.1f}.npz'.format(project_name, load_name, dP, dQ)
blob = np.load(os.path.join(data_dir, data_file), allow_pickle=True)
time = blob['time']
data = blob['data'].item()
speed = data['gen']['s:xspeed']
power = data['gen']['s:pgt']
device_names = blob['device_names'].item()
gen_names = device_names['gen']
n_generators = power.shape[1]
t_event = 10. # [s]

In [ ]:
attributes = blob['attributes'].item()
S = np.array(attributes['gen']['typ_id.sgn'])

before = np.where(time < t_event)[0][-1]
ΔP = np.squeeze(np.diff(power[[before,-1],:], axis=0))
ΔPgen = S @ ΔP

P,Q = data['load']['m:Psum:bus1'], data['load']['m:Qsum:bus1']
ΔPload = P[-1] - P[0]
ratio = ΔPload/P[0]*100
print('ΔP load: {:.2f} MW ({}{:.2f}%).'.format(ΔPload, '+' if ratio > 0 else '', ratio))
print('ΔP generators: {:.2f} MW.'.format(ΔPgen))

In [ ]:
fig,ax = plt.subplots(2, 1, sharex=True, figsize=(6,4))
ax[0].plot(time, P, 'k', lw=1)
ax[1].plot(time, Q, 'r', lw=1)
ax[1].set_xlabel('Time [s]')
ax[0].set_ylabel('P [MW]')
ax[1].set_ylabel('Q [MVAr]')
sns.despine()

In [ ]:
cols = 3
rows = n_generators // cols
fig,ax = plt.subplots(rows, cols, figsize=(cols*3, rows*1.5), sharex=True, sharey=True)
idx, = np.where(time > t_event - 2)
for k in range(n_generators):
    i,j = k//cols, k%cols
    ax[i,j].plot(time[idx[[0,-1]]], speed[idx[0],k]+np.zeros(2), 'r--')
    ax[i,j].plot(time[idx], speed[idx,k], 'k', lw=1)
    ax[i,j].set_title(gen_names[k], fontsize=9)
for a in ax[-1,:]:
    a.set_xlabel('Time [s]')
#     a.set_xticks(np.r_[t_event : time[-1] : 20])
for a in ax[:,0]:
    a.set_ylabel('ω [p.u.]')
sns.despine()
fig.tight_layout()

In [ ]:
cols = 3
rows = n_generators // cols
fig,ax = plt.subplots(rows, cols, figsize=(cols*3, rows*1.5), sharex=True)
idx, = np.where(time > t_event - 2)
for k in range(n_generators):
    i,j = k//cols, k%cols
    ax[i,j].plot(time[idx[[0,-1]]], power[idx[0],k]+np.zeros(2), 'r--')
    ax[i,j].plot(time[idx], power[idx,k], 'k', lw=1)
    ax[i,j].set_title(gen_names[k], fontsize=9)
for a in ax[-1,:]:
    a.set_xlabel('Time [s]')
#     a.set_xticks(np.r_[t_event : time[-1] : 20])
for a in ax[:,0]:
    a.set_ylabel('Power [p.u.]')
sns.despine()
fig.tight_layout()